In [10]:
!pip install transformers faiss-cpu torch fitz


  Using cached faiss_cpu-1.10.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (4.4 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-

In [5]:
!pip uninstall pymupdf
! pip install pymupdf


Found existing installation: PyMuPDF 1.25.5
Uninstalling PyMuPDF-1.25.5:
  Would remove:
    /usr/local/bin/pymupdf
    /usr/local/lib/python3.11/dist-packages/fitz/*
    /usr/local/lib/python3.11/dist-packages/pymupdf-1.25.5.dist-info/*
    /usr/local/lib/python3.11/dist-packages/pymupdf/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.11/dist-packages/fitz/.DS_Store
    /usr/local/lib/python3.11/dist-packages/fitz/frontend.py
    /usr/local/lib/python3.11/dist-packages/fitz/tools/__init__.py
    /usr/local/lib/python3.11/dist-packages/fitz/tools/builder.py
    /usr/local/lib/python3.11/dist-packages/fitz/tools/commandline.py
    /usr/local/lib/python3.11/dist-packages/fitz/tools/graphutils.py
    /usr/local/lib/python3.11/dist-packages/fitz/tools/setup_project.py
Proceed (Y/n)? y
  Successfully uninstalled PyMuPDF-1.25.5
  Using cached pymupdf-1.25.5-cp39-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (3.4 kB)
Using cached pymupdf-1.25.5-cp39-a

In [2]:
# Install required libraries (make sure you run these in Colab first)
!pip install transformers torch fitz


In [11]:
import torch
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
import fitz  # PyMuPDF

import warnings
warnings.filterwarnings("ignore")

import time
import textwrap

qa_pipeline = pipeline("question-answering")

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set pad_token to eos_token

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    return " ".join([page.get_text() for page in doc])

# Break the full text into chunks to avoid token overflow
def chunk_text(text, chunk_size=500):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

# Generate an answer from a chunk using GPT-2
def answer_question_with_qa_model(question, context_chunk):
    try:
        result = qa_pipeline(question=question, context=context_chunk)
        return result['answer']
    except:
        return ""


    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Only return what's after "Answer:"
    return generated_text.split("Answer:")[-1].strip()

# Chatbot class that uses all the above
class PDFChatbot:
    def __init__(self, pdf_path):
        print("🔍 Loading and processing PDF...")
        self.text_chunks = chunk_text(extract_text_from_pdf(pdf_path))
        print(f"✅ Loaded {len(self.text_chunks)} chunks.")

    def chat(self, question):
      best_response = ""
      best_score = 0

      for chunk in self.text_chunks:
        response = answer_question_with_qa_model(question, chunk)
        score = len(response.strip())

        if score > best_score:
            best_score = score
            best_response = response

      return best_response or "Sorry, I couldn't find an answer in the PDF."


# === Run the chatbot ===
pdf_path = "/content/Seizure Detection and Probability Prediction using Random Forests.pdf"
chatbot = PDFChatbot(pdf_path)

def pretty_print_response(response):
    print("\nChatbot 🤖: ", end="")
    for char in response:
        print(char, end="", flush=True)
        time.sleep(0.01)  # Typing effect
    print("\n")  # Add a new line at the end for spacing

print("\n✨ Chatbot is ready to answer your questions! Type 'exit', 'quit', or 'stop' to end.\n")

while True:
    user_input = input("🧠 You: ")
    if user_input.lower() in ["exit", "quit", "stop"]:
        print("Chatbot 🤖: Goodbye! 👋")
        break
    response = chatbot.chat(user_input)

    # Wrap long lines (optional)
    wrapped_response = textwrap.fill(response, width=80)
    pretty_print_response(wrapped_response)

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


🔍 Loading and processing PDF...
✅ Loaded 36 chunks.

✨ Chatbot is ready to answer your questions! Type 'exit', 'quit', or 'stop' to end.

🧠 You: what is the paper about?

Chatbot 🤖: The prediction module of the project also helps identify the  probability of
seizure onset

🧠 You: what is new about this?

Chatbot 🤖: scores achieved in cross-validation varied  a significant amount from
leaderboard scores

🧠 You: explain in brief the paper

Chatbot 🤖: scores achieved in cross-validation varied  a significant amount from
leaderboard scores

🧠 You: what is the method used in the paper

Chatbot 🤖: Random Forest classifier for EEG-based seizure prediction

🧠 You: is there maths in it?

Chatbot 🤖: It turned out to give  a better result than using other frequency ranges

🧠 You: give some specifics about the paper

Chatbot 🤖: Combinations of multiple features eventually proved to  provide a better
classification score

🧠 You: what is the data like?

Chatbot 🤖: Combinations of multiple features e